### Gerenciamento do diretório do projeto

In [1]:
import sys
import os
script_dir = os.getcwd() # coleata o diretorio do projeto atual
sys.path.append(script_dir)

### Imports necessários para o uso do Gemini

In [2]:
import pandas as pd
from time import sleep
import pickle
from dotenv import load_dotenv, find_dotenv
import google.generativeai as gemini
import google.generativeai.client as client

### Gerenciamento da Key necessária para uso da API do Gemini

Para uma melhor modularização do código num geral, decidimos usar funções que serão chamadas no decorrer do código

`carrega_chave()`: Carrega a chave de API do Google a partir de um arquivo .env e a configura para uso.

`verifica_chave()`: Verifica se o arquivo .env existe e retorna seu caminho.

In [3]:
def carrega_chave(): # Carrega uma chave do arquivo .env
    _ = load_dotenv(find_dotenv())
    chave = os.getenv("GOOGLE_API_KEY")
    gemini.configure(api_key=os.getenv('GOOGLE_API_KEY'))

    print(f"Chave API carregada com sucesso!")
    return chave

def verifica_chave(): # Verifica a existência de um arquivo .env 
    return find_dotenv()


In [4]:
chave = carrega_chave()

Chave API carregada com sucesso!


In [5]:
# Carregando o modelo
model = gemini.GenerativeModel('gemini-1.5-flash')

### Carregando a base de dados que contém tanto o enunciado quando as alternativas em uma única String

In [6]:
data = pd.read_csv(r'CSV\alternativas_separadas_pp.csv')
data.head(5)

,Ano,Enunciado,Alternativa_A,Alternativa_B,Alternativa_C,Alternativa_D,Alternativa_E,Area_de_Conhecimento
0,2020,bienalsur una invitación cruz la front ded señ...,nov cri propósit dea apont divers obr artíst e...,inspir transform econôm política país europ,acus artist expõ obr local pouc tradic,promov social difer artist popul geral,realiz expos artíst sobr event espor jog olímp,Linguagens
1,2020,españ corazón feder dim una gran sorpr hab pre...,eder garcí lorc ness episódi uso express alimó...,admir tour manifest discurs escri,surpr discurs escri receb públic,conhec escri acerc tradic tour espanhol,perspicác escri discurs inspir form tour,Linguagens
2,2020,dispon httpinversorsaludcom acess 18 ago 2017 ...,condicionando exerc ativ comunitár,ora incluindos açõ ora ordenando inform,ora instruindo ato ora reprimindo falh,adicion voz post diverg açõ mor,impondos voz autoridad órg governament,Linguagens
3,2020,lo orígen habit expresíon che hay alg má argen...,xpress reflet quant imigr usu vocábul,perspec filólog uso dess palavr,identific argentin palavr che,divers form dess variedad castelh,impos língu espanhol sobr língu indígen,Linguagens
4,2020,habl con acent andaluz telediari la 9 hac uno ...,ora acord discuss autor defend soberan fal sob...,estranh percep fal andaluz,lut andaluz divers linguís,hegemon sotaqu bas prestígi soc,vis estereotip própri andaluz acerc fal,Linguagens


### Função que faz a requisição para a API do Gemini

Contém um simples prompt a ser enviado ao Gemini, cujo consiste em explicar a classificação a ser feita antes de passar a questão, o Gemini então, retorna uma string numérica de 0 a 3 de acordo com a classificação da questão:

`0 = Linguagens`

`1 = Ciências Humanas`

`2 = Ciências da Natuerza`

`3 = Matemática`


In [7]:
def envia_geminho(questao):
    return model.generate_content(f"você receberá uma questão de uma prova, quero que você classifique entre Linguagens = 0, Ciências Humanas = 1, Ciências da Natureza = 2, Matemática = 3, lembrando que para considerar linguagens pode ser uma pergunta de inglês, português ou espanhol. Para me fornecer a saída, basta unicamente escrever o número correspondente. A questão é: {questao}")

### Mandando as perguntas para o Gemini

`enum_alt` é a variável que guarda apenas a String contendo o enunciado e as alternativas citado anteriormente.

In [8]:
enum_alt = data["Enunciado"]
len(enum_alt)

792

`numero_req` é a variável que limita em 5 requisições de forma que, independente do tamanho da base de entrada, ainda serão 5 requisições não abrindo brechas para quebra de código.

In [9]:
mumero_req = len(enum_alt) // 5 + 1  if len(enum_alt) % 5 != 0 else len(enum_alt) // 5
mumero_req

159

Essa label é quem faz efetivamente a requisição, ela funciona de acordo com as características da API do gemini flash usado para esse projeto.

De modo que são feitas 15 requisições por minuto até totalizar toda a base de input.

In [10]:
respostas_anteriores = []
contador = 0

for questao in enum_alt:
    if contador == 15:
        contador = 0
        sleep(60)
    
    resp = envia_geminho(questao)
    
    contador += 1
    respostas_anteriores.append(resp)

E por fim, é salvo com uso da biblioteca pickle, as respostas dadas pela API do Gemini em `lista_1a1.pkl`

In [11]:
# Salvando a lista em um arquivo
with open('lista_enum_1a1_pp.pkl', 'wb') as file:
    pickle.dump(respostas_anteriores, file)